In [5]:
# Full Jupyter Notebook Code: Sewer Event Detection with Isolation Forest

import pandas as pd
from sklearn.ensemble import IsolationForest
import joblib

# Load dataset
df = pd.read_csv("sewer_only_labeled.csv")  # Adjust path if running locally

# Step 1: Filter normal data (ammonia <= 100) for training
normal_data = df[df['ammonia'] <= 100]

# Step 2: Select features
features = ['ammonia', 'co2', 'humidity', 'temperature', 'visitor',
            'iaq', 'delta_ammonia', 'rolling_avg_ammonia',
            'rolling_std_ammonia', 'IAQ_proxy', 'hour']
X = normal_data[features]

# Step 3: Train Isolation Forest
model = IsolationForest(contamination=0.05, random_state=42)
model.fit(X)

# Step 4: Save model
joblib.dump(model, "isolation_forest_sewer_model.pkl")

# Step 5: Example new input (simulate real-time reading)
new_data = {
    'ammonia': 180,
    'co2': 500,
    'humidity': 85,
    'temperature': 28,
    'visitor': 15,
    'iaq': 115,
    'delta_ammonia': 30,
    'rolling_avg_ammonia': 170,
    'rolling_std_ammonia': 20,
    'IAQ_proxy': 113,
    'hour': 3
}

# Step 6: Convert to DataFrame
input_df = pd.DataFrame([new_data])

# Step 7: Predict sewer event
prediction = model.predict(input_df)[0]  # -1 = anomaly, 1 = normal
score = model.decision_function(input_df)[0]  # Lower = more anomalous

# Step 8: Display result
result = {
    "prediction": "🚨 Sewer Event" if prediction == -1 else "✅ Normal",
    "anomaly_score": float(score),
    "sensor_readings": new_data
}

# Step 9: Optional log to CSV
log = input_df.copy()
log['prediction'] = result["prediction"]
log['anomaly_score'] = result["anomaly_score"]
log.to_csv("sewer_event_log.csv", mode='a', header=not pd.io.common.file_exists("/mnt/data/sewer_event_log.csv"), index=False)

# Show result
result



{'prediction': '🚨 Sewer Event',
 'anomaly_score': -0.06978223258219063,
 'sensor_readings': {'ammonia': 180,
  'co2': 500,
  'humidity': 85,
  'temperature': 28,
  'visitor': 15,
  'iaq': 115,
  'delta_ammonia': 30,
  'rolling_avg_ammonia': 170,
  'rolling_std_ammonia': 20,
  'IAQ_proxy': 113,
  'hour': 3}}

In [8]:
# toilet_dashboard_sewer_ml.py
import streamlit as st
import pandas as pd
import joblib
from datetime import datetime
import os
import requests

# Load Isolation Forest model
model = joblib.load("isolation_forest_sewer_model.pkl")

# Log file path
LOG_FILE = "sewer_event_log.csv"

# Telegram settings (replace with your own)
TELEGRAM_BOT_TOKEN = "<8014356172:AAFZWleta0QmFjdTu-5WfrobVxahR5kI0Qw>"
TELEGRAM_CHAT_ID = "< 850047031>"
TELEGRAM_API = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"

# Streamlit UI
st.set_page_config(page_title="Sewer Event Monitor", layout="wide")
st.title("🚽 Sewer Event Detection Dashboard")

st.sidebar.header("Sensor Input")

# Input fields
sensor_input = {
    'ammonia': st.sidebar.number_input("Ammonia (ppm)", 0.0, 2000.0, step=1.0),
    'co2': st.sidebar.number_input("CO2", 0, 10000, step=50),
    'humidity': st.sidebar.number_input("Humidity (%)", 0.0, 100.0, step=0.1),
    'temperature': st.sidebar.number_input("Temperature (°C)", 0.0, 50.0, step=0.1),
    'visitor': st.sidebar.number_input("Visitor Count", 0, 1000, step=1),
    'iaq': st.sidebar.number_input("IAQ Index", 0.0, 500.0, step=1.0),
    'delta_ammonia': st.sidebar.number_input("Delta Ammonia", 0.0, 200.0, step=0.1),
    'rolling_avg_ammonia': st.sidebar.number_input("Rolling Avg Ammonia", 0.0, 2000.0, step=1.0),
    'rolling_std_ammonia': st.sidebar.number_input("Rolling Std Ammonia", 0.0, 200.0, step=1.0),
    'IAQ_proxy': st.sidebar.number_input("IAQ Proxy (Temp + Humidity)", 0.0, 200.0, step=1.0),
    'hour': st.sidebar.slider("Hour of Day", 0, 23, 12)
}

# Process prediction
if sensor_input['ammonia'] > 100:
    input_df = pd.DataFrame([sensor_input])
    prediction = model.predict(input_df)[0]  # -1 = anomaly
    score = model.decision_function(input_df)[0]
    is_sewer = prediction == -1

    st.subheader("🔍 Prediction Result")
    st.markdown(f"**Prediction:** {'🚨 Sewer Event' if is_sewer else '✅ Normal'}")
    st.markdown(f"**Anomaly Score:** {score:.4f}")

    st.markdown("### 📊 Sensor Readings")
    st.write(pd.DataFrame([sensor_input]))

    # Action taken
    if st.button("✅ Mark as Resolved"):
        log_entry = input_df.copy()
        log_entry['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry['prediction'] = 'Sewer Event' if is_sewer else 'Normal'
        log_entry['anomaly_score'] = score

        file_exists = os.path.exists(LOG_FILE)
        log_entry.to_csv(LOG_FILE, mode='a', header=not file_exists, index=False)

        # Telegram alert
        msg = f"✅ Sewer Event Resolved\nTimestamp: {log_entry['timestamp'].values[0]}\nAnomaly Score: {score:.4f}"
        sensor_lines = "\n".join([f"{k}: {v}" for k, v in sensor_input.items()])
        msg += f"\n\n📟 Sensor Readings:\n{sensor_lines}"

        if TELEGRAM_BOT_TOKEN != "<YOUR_BOT_TOKEN>":
            requests.post(TELEGRAM_API, data={"chat_id": TELEGRAM_CHAT_ID, "text": msg})

        st.success("Sewer event marked as resolved and logged.")

else:
    st.info("Ammonia level too low to trigger sewer detection.")


2025-06-17 01:09:36.669 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 01:09:36.671 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 01:09:36.672 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 01:09:36.674 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 01:09:36.675 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 01:09:36.677 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 01:09:36.678 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 01:09:36.679 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar